In [2]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 3.9 MB/s eta 0:00:0000:0100:01


## 1. Preprocesar imagenes satelitales a RGB

In [4]:
import os
import numpy as np
import rasterio
from PIL import Image

# Directorio de entrada y salida
input_dir = "sample_data/SatImages/perusat_v5"
output_dir = "sample_data/SatImages/perusat_v5_rgb/HR"

# Crear el directorio de salida si no existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Recorrer los archivos con extensión .tif
for archivo in os.listdir(input_dir):
    if archivo.lower().endswith(".tif"):
        ruta_imagen = os.path.join(input_dir, archivo)
        with rasterio.open(ruta_imagen) as src:
            # Se asume que la imagen tiene al menos 3 bandas: R, G, B
            # Si la imagen tuviera más bandas o el orden fuera distinto, habría que ajustarlo.
            red = src.read(1)
            green = src.read(2)
            blue = src.read(3)
            # Combinar las tres bandas en un solo array (formando una imagen RGB)
            rgb = np.dstack((red, green, blue))
            
            # Si la imagen no está en el rango 0-255, se realiza una normalización sencilla.
            if rgb.dtype != np.uint8:
                rgb_min, rgb_max = np.min(rgb), np.max(rgb)
                # Evitar división por cero
                if rgb_max != rgb_min:
                    rgb = ((rgb - rgb_min) / (rgb_max - rgb_min) * 255).astype(np.uint8)
                else:
                    rgb = np.zeros_like(rgb, dtype=np.uint8)

        # Guardar la imagen en formato PNG
        imagen_rgb = Image.fromarray(rgb)
        nombre_salida = os.path.splitext(archivo)[0] + ".png"
        ruta_salida = os.path.join(output_dir, nombre_salida)
        imagen_rgb.save(ruta_salida)
        print(f"Guardado: {ruta_salida}")

Guardado: sample_data/SatImages/perusat_v5_rgb/HR/IMG_PER1_20190417152356_ORT_MS_000041_3584-3584.png
Guardado: sample_data/SatImages/perusat_v5_rgb/HR/IMG_PER1_20170422154946_ORT_MS_001895_1536-4096.png
Guardado: sample_data/SatImages/perusat_v5_rgb/HR/IMG_PER1_20190422153440_ORT_MS_000041_1536-4096.png
Guardado: sample_data/SatImages/perusat_v5_rgb/HR/IMG_PER1_20190318155341_ORT_MS_006529_2048-3072.png
Guardado: sample_data/SatImages/perusat_v5_rgb/HR/IMG_PER1_20170422154946_ORT_MS_000659_1024-512.png
Guardado: sample_data/SatImages/perusat_v5_rgb/HR/IMG_PER1_20170422154946_ORT_MS_001895_4608-1536.png
Guardado: sample_data/SatImages/perusat_v5_rgb/HR/IMG_PER1_20190703144250_ORT_MS_000672_2048-4096.png
Guardado: sample_data/SatImages/perusat_v5_rgb/HR/IMG_PER1_20190318155341_ORT_MS_008563_2048-4096.png
Guardado: sample_data/SatImages/perusat_v5_rgb/HR/IMG_PER1_20170422154946_ORT_MS_000041_5120-2048.png
Guardado: sample_data/SatImages/perusat_v5_rgb/HR/IMG_PER1_20190318155341_ORT_MS_00

In [5]:
import os
from PIL import Image

# Directorio de entrada (imágenes RGB en HR)
hr_dir = "sample_data/SatImages/perusat_v5_rgb/HR"

# Directorio base para las imágenes de menor resolución (LR_BICUBIC)
lr_base_dir = "sample_data/SatImages/perusat_v5_rgb/LR_BICUBIC"
x2_dir = os.path.join(lr_base_dir, "X2")  # Mitad de resolución
x4_dir = os.path.join(lr_base_dir, "X4")  # Un cuarto de resolución

# Crear las carpetas de salida si no existen
for folder in [lr_base_dir, x2_dir, x4_dir]:
    if not os.path.exists(folder):
        os.makedirs(folder)

# Procesar cada imagen PNG en la carpeta HR
for archivo in os.listdir(hr_dir):
    if archivo.lower().endswith(".png"):
        ruta_imagen = os.path.join(hr_dir, archivo)
        imagen = Image.open(ruta_imagen)

        # Obtener dimensiones originales
        ancho, alto = imagen.size

        # Generar versión con la mitad de resolución (X2)
        nuevo_tamano_x2 = (ancho // 2, alto // 2)
        imagen_x2 = imagen.resize(nuevo_tamano_x2, resample=Image.BICUBIC)
        nombre_base = os.path.splitext(archivo)[0]
        nombre_salida_x2 = nombre_base + "_x2.png"
        ruta_salida_x2 = os.path.join(x2_dir, nombre_salida_x2)
        imagen_x2.save(ruta_salida_x2)
        print(f"Guardado (X2): {ruta_salida_x2}")

        # Generar versión con un cuarto de resolución (X4)
        nuevo_tamano_x4 = (ancho // 4, alto // 4)
        imagen_x4 = imagen.resize(nuevo_tamano_x4, resample=Image.BICUBIC)
        nombre_salida_x4 = nombre_base + "_x4.png"
        ruta_salida_x4 = os.path.join(x4_dir, nombre_salida_x4)
        imagen_x4.save(ruta_salida_x4)
        print(f"Guardado (X4): {ruta_salida_x4}")


Guardado (X2): sample_data/SatImages/perusat_v5_rgb/LR_BICUBIC/X2/IMG_PER1_20180926153436_ORT_MS_000659_1024-3584_x2.png
Guardado (X4): sample_data/SatImages/perusat_v5_rgb/LR_BICUBIC/X4/IMG_PER1_20180926153436_ORT_MS_000659_1024-3584_x4.png
Guardado (X2): sample_data/SatImages/perusat_v5_rgb/LR_BICUBIC/X2/IMG_PER1_20190703144250_ORT_MS_000672_1024-4096_x2.png
Guardado (X4): sample_data/SatImages/perusat_v5_rgb/LR_BICUBIC/X4/IMG_PER1_20190703144250_ORT_MS_000672_1024-4096_x4.png
Guardado (X2): sample_data/SatImages/perusat_v5_rgb/LR_BICUBIC/X2/IMG_PER1_20181213153432_ORT_MS_000659_1536-2560_x2.png
Guardado (X4): sample_data/SatImages/perusat_v5_rgb/LR_BICUBIC/X4/IMG_PER1_20181213153432_ORT_MS_000659_1536-2560_x4.png
Guardado (X2): sample_data/SatImages/perusat_v5_rgb/LR_BICUBIC/X2/IMG_PER1_20190311155633_ORT_MS_000041_3072-3584_x2.png
Guardado (X4): sample_data/SatImages/perusat_v5_rgb/LR_BICUBIC/X4/IMG_PER1_20190311155633_ORT_MS_000041_3072-3584_x4.png
Guardado (X2): sample_data/SatIm

In [1]:
import os
import glob
import rasterio

# Directorio donde están los .tif originales
tif_dir = "sample_data/SatImages/perusat_v5"
tif_files = glob.glob(os.path.join(tif_dir, "*.tif"))

# Listas para acumular datos
resolutions = []
layer_counts = []
sizes_mb = []

for fp in tif_files:
    # Abrir con rasterio para leer metadatos
    with rasterio.open(fp) as src:
        # resolución (ancho x alto)
        resolutions.append((src.width, src.height))
        # número de capas (bands)
        layer_counts.append(src.count)
    # tamaño en megabytes
    size = os.path.getsize(fp) / (1024 * 1024)
    sizes_mb.append(size)

# 1. ¿Cuántos archivos?
num_files = len(tif_files)

# 2. Resolución
unique_res = set(resolutions)
if len(unique_res) == 1:
    res_str = f"{resolutions[0][0]}×{resolutions[0][1]}"
else:
    # definir "menor" y "mayor" según área (width*height)
    areas = [(w*h, w, h) for w, h in unique_res]
    min_area, min_w, min_h = min(areas)
    max_area, max_w, max_h = max(areas)
    res_str = f"{min_w}×{min_h} - {max_w}×{max_h}"

# 3. Número de capas
min_layers = min(layer_counts)
max_layers = max(layer_counts)
if min_layers == max_layers:
    layers_str = str(min_layers)
else:
    layers_str = f"{min_layers} - {max_layers}"

# 4. Tamaño en MB
min_size = min(sizes_mb)
max_size = max(sizes_mb)
if abs(min_size - max_size) < 1e-6:
    size_str = f"{min_size:.2f} MB"
else:
    size_str = f"{min_size:.2f} MB - {max_size:.2f} MB"

# Mostrar resultado
print(f"■ Número de archivos: {num_files}")
print(f"■ Resolución: {res_str}")
print(f"■ Número de capas: {layers_str}")
print(f"■ Tamaño (MB): {size_str}")


■ Número de archivos: 2937
■ Resolución: 512×512
■ Número de capas: 4
■ Tamaño (MB): 2.00 MB


In [3]:
import os
import glob
from PIL import Image

# Directorio donde están los .png
png_dir = "sample_data/DF2K_train_HR"
png_files = glob.glob(os.path.join(png_dir, "*.png"))

# Listas para acumular datos
resolutions = []
channel_counts = []
sizes_mb = []

for fp in png_files:
    # Abrir con PIL para leer metadatos
    with Image.open(fp) as img:
        # resolución (ancho x alto)
        resolutions.append((img.width, img.height))
        # número de canales/bandas
        channel_counts.append(len(img.getbands()))
    # tamaño en megabytes
    size = os.path.getsize(fp) / (1024 * 1024)
    sizes_mb.append(size)

# 1. ¿Cuántos archivos?
num_files = len(png_files)

# 2. Resolución
unique_res = set(resolutions)
if len(unique_res) == 1:
    res_str = f"{resolutions[0][0]}×{resolutions[0][1]}"
else:
    # ordenar por área (width*height)
    areas = [(w*h, w, h) for w, h in unique_res]
    min_area, min_w, min_h = min(areas)
    max_area, max_w, max_h = max(areas)
    res_str = f"{min_w}×{min_h} - {max_w}×{max_h}"

# 3. Número de canales
min_ch = min(channel_counts)
max_ch = max(channel_counts)
if min_ch == max_ch:
    channels_str = str(min_ch)
else:
    channels_str = f"{min_ch} - {max_ch}"

# 4. Tamaño en MB
min_size = min(sizes_mb)
max_size = max(sizes_mb)
if abs(min_size - max_size) < 1e-6:
    size_str = f"{min_size:.2f} MB"
else:
    size_str = f"{min_size:.2f} MB - {max_size:.2f} MB"

# Mostrar resultado
print(f"■ Número de archivos: {num_files}")
print(f"■ Resolución: {res_str}")
print(f"■ Número de canales: {channels_str}")
print(f"■ Tamaño (MB): {size_str}")


■ Número de archivos: 3450
■ Resolución: 1584×816 - 2040×2040
■ Número de canales: 3
■ Tamaño (MB): 1.05 MB - 8.67 MB


In [4]:
png_dir = "sample_data/DF2K_valid_HR"
png_files = glob.glob(os.path.join(png_dir, "*.png"))

# Listas para acumular datos
resolutions = []
channel_counts = []
sizes_mb = []

for fp in png_files:
    # Abrir con PIL para leer metadatos
    with Image.open(fp) as img:
        # resolución (ancho x alto)
        resolutions.append((img.width, img.height))
        # número de canales/bandas
        channel_counts.append(len(img.getbands()))
    # tamaño en megabytes
    size = os.path.getsize(fp) / (1024 * 1024)
    sizes_mb.append(size)

# 1. ¿Cuántos archivos?
num_files = len(png_files)

# 2. Resolución
unique_res = set(resolutions)
if len(unique_res) == 1:
    res_str = f"{resolutions[0][0]}×{resolutions[0][1]}"
else:
    # ordenar por área (width*height)
    areas = [(w*h, w, h) for w, h in unique_res]
    min_area, min_w, min_h = min(areas)
    max_area, max_w, max_h = max(areas)
    res_str = f"{min_w}×{min_h} - {max_w}×{max_h}"

# 3. Número de canales
min_ch = min(channel_counts)
max_ch = max(channel_counts)
if min_ch == max_ch:
    channels_str = str(min_ch)
else:
    channels_str = f"{min_ch} - {max_ch}"

# 4. Tamaño en MB
min_size = min(sizes_mb)
max_size = max(sizes_mb)
if abs(min_size - max_size) < 1e-6:
    size_str = f"{min_size:.2f} MB"
else:
    size_str = f"{min_size:.2f} MB - {max_size:.2f} MB"

# Mostrar resultado
print(f"■ Número de archivos: {num_files}")
print(f"■ Resolución: {res_str}")
print(f"■ Número de canales: {channels_str}")
print(f"■ Tamaño (MB): {size_str}")

■ Número de archivos: 100
■ Resolución: 2040×816 - 2040×2040
■ Número de canales: 3
■ Tamaño (MB): 0.85 MB - 6.74 MB


## 2. Preprocesar imagenes satelitales

In [17]:
import os
import cv2
import tifffile as tiff
from tqdm import tqdm
import sys  # Para dirigir la barra de progreso a stdout

def create_dirs(base_dir):
    """
    Crea la estructura de carpetas necesaria:
        base_dir/
        ├── HR
        └── LR_BICUBIC
            ├── X2
            └── X4
    """
    hr_dir = os.path.join(base_dir, "HR")
    lr_bicubic_dir = os.path.join(base_dir, "LR_BICUBIC")
    lr_x2_dir = os.path.join(lr_bicubic_dir, "X2")
    lr_x4_dir = os.path.join(lr_bicubic_dir, "X4")

    os.makedirs(hr_dir, exist_ok=True)
    os.makedirs(lr_x2_dir, exist_ok=True)
    os.makedirs(lr_x4_dir, exist_ok=True)

    return hr_dir, lr_x2_dir, lr_x4_dir

def process_images(input_dir):
    """
    Procesa cada imagen .tif en input_dir:
    1. Lee la imagen con tifffile para conservar las 4 bandas (RGB + NIR).
    2. Guarda la imagen original en HR.
    3. Genera versiones LR bicúbicas x2 y x4 (manteniendo las 4 bandas) y las guarda en las carpetas correspondientes.
    Muestra una barra de progreso en modo texto (ascii) para evitar dependencias de widgets.
    """
    hr_dir, lr_x2_dir, lr_x4_dir = create_dirs(input_dir)
    all_files = [f for f in os.listdir(input_dir) if f.lower().endswith(".tif")]

    for filename in tqdm(all_files,
                         desc="Procesando imágenes .tif",
                         unit="imagen",
                         ascii=True,
                         file=sys.stdout):
        img_path = os.path.join(input_dir, filename)

        try:
            # Leer la imagen conservando todas las bandas (shape: H x W x 4)
            img = tiff.imread(img_path)
        except Exception as e:
            print(f"Error leyendo {filename}: {e}")
            continue

        # Verificar que la imagen tenga 4 dimensiones (4 bandas)
        if img.ndim != 3 or img.shape[2] != 4:
            print(f"Ignorando {filename}, no tiene 4 bandas (shape={img.shape})")
            continue

        name, ext = os.path.splitext(filename)

        # Guardar la imagen original en HR usando tifffile
        hr_output_path = os.path.join(hr_dir, filename)
        try:
            tiff.imwrite(hr_output_path, img)
        except Exception as e:
            print(f"No se pudo guardar HR de {filename}: {e}")
            continue

        height, width, bands = img.shape

        # Función interna para redimensionar manteniendo 4 bandas
        def resize_multiband(image, new_size):
            """
            Recibe un arreglo NumPy con forma (H, W, C) y lo redimensiona con cv2,
            preservando las C bandas.
            """
            # OpenCV redimensiona arrays con forma (H, W, C) correctamente
            return cv2.resize(image, new_size, interpolation=cv2.INTER_CUBIC)

        # Generar y guardar versión x2 (256x256)
        new_size_x2 = (width // 2, height // 2)
        lr_x2 = resize_multiband(img, new_size_x2)
        lr_x2_filename = f"{name}_x2{ext}"
        lr_x2_output_path = os.path.join(lr_x2_dir, lr_x2_filename)
        try:
            tiff.imwrite(lr_x2_output_path, lr_x2)
        except Exception as e:
            print(f"No se pudo guardar LR x2 de {filename}: {e}")

        # Generar y guardar versión x4 (128x128)
        new_size_x4 = (width // 4, height // 4)
        lr_x4 = resize_multiband(img, new_size_x4)
        lr_x4_filename = f"{name}_x4{ext}"
        lr_x4_output_path = os.path.join(lr_x4_dir, lr_x4_filename)
        try:
            tiff.imwrite(lr_x4_output_path, lr_x4)
        except Exception as e:
            print(f"No se pudo guardar LR x4 de {filename}: {e}")

if __name__ == "__main__":
    # Ruta de la carpeta que contiene las imágenes .tif originales
    base_folder = r"Datasets\SatImages\perusat_v5"
    process_images(base_folder)

Procesando imágenes .tif: 100%|##########| 2937/2937 [00:10<00:00, 291.04imagen/s]


In [8]:
import numpy as np
import rasterio


def compute_ranges_per_band_from_array(image_array: np.ndarray) -> dict:
    """
    Calcula el valor mínimo, máximo y rango (max - min) de cada banda
    de una imagen representada como un array de NumPy.

    Args:
        image_array (H, W, C)-ndarray: imagen con C capas en la última dimensión.

    Returns:
        dict: diccionario con claves 'band_1', 'band_2', … 'band_C', cada una
              conteniendo {'min': ..., 'max': ..., 'range': ...}.
    """
    band_ranges = {}
    num_bands = image_array.shape[-1]  # número de capas (por ejemplo, R, G, B, NIR, F)

    for b in range(num_bands):
        band_data = image_array[..., b]
        # Obtener valor mínimo y máximo de la capa b
        min_val = float(np.min(band_data))
        max_val = float(np.max(band_data))
        band_ranges[f'band_{b + 1}'] = {
            'min': min_val,
            'max': max_val,
            'range': max_val - min_val
        }

    return band_ranges


def compute_ranges_per_band_from_file(image_path: str) -> dict:
    """
    Lee una imagen multibanda desde disco (por ejemplo, GeoTIFF con RGB+NIR+F),
    y calcula el valor mínimo, máximo y rango de cada banda.

    Args:
        image_path (str): ruta al archivo de imagen (GeoTIFF u otro formato soportado por rasterio).

    Returns:
        dict: diccionario con claves 'band_1', 'band_2', … 'band_N', cada una
              conteniendo {'min': ..., 'max': ..., 'range': ...}.
    """
    band_ranges = {}
    # Abrir el archivo con rasterio para leer todas las bandas
    with rasterio.open(image_path) as src:
        # src.read() devuelve un array de forma (bands, height, width)
        image_stack = src.read()

    num_bands = image_stack.shape[0]  # número de capas

    for b in range(num_bands):
        band_data = image_stack[b]  # capa b (2D: height × width)
        # Obtener valor mínimo y máximo de la capa b
        min_val = float(np.min(band_data))
        max_val = float(np.max(band_data))
        band_ranges[f'band_{b + 1}'] = {
            'min': min_val,
            'max': max_val,
            'range': max_val - min_val
        }

    return band_ranges


# ============================
# Ejemplo de uso
# ============================
if __name__ == "__main__":
    # Ejemplo 1: si ya tienes la imagen cargada como un array de NumPy
    # Supongamos que 'img_array' es un ndarray de forma (H, W, 5) para RGB+NIR+F
    # img_array = np.random.randint(0, 4096, size=(512, 512, 5), dtype=np.uint16)
    # ranges_dict = compute_ranges_per_band_from_array(img_array)
    # print("Rangos por banda (desde array):", ranges_dict)

    # Ejemplo 2: cargar directamente un archivo multibanda (por ejemplo GeoTIFF)
    image_path = "Datasets/SatImages/perusat_v5/HR/IMG_PER1_20161203152919_ORT_MS_000670_1024-512.tif"
    ranges_dict_file = compute_ranges_per_band_from_file(image_path)
    print("Rangos por banda (desde archivo):", ranges_dict_file)

Rangos por banda (desde archivo): {'band_1': {'min': 238.0, 'max': 1177.0, 'range': 939.0}, 'band_2': {'min': 289.0, 'max': 999.0, 'range': 710.0}, 'band_3': {'min': 386.0, 'max': 987.0, 'range': 601.0}, 'band_4': {'min': 201.0, 'max': 1488.0, 'range': 1287.0}}


In [19]:
import rasterio

def inspect_raster(path: str):
    """Imprime metadatos básicos de un GeoTIFF (número de bandas, tamaño, sistema de coordenadas)."""
    with rasterio.open(path) as src:
        print("Ruta del archivo:", path)
        print("Número de bandas:", src.count)
        print("Anchura (px):", src.width)
        print("Altura (px):", src.height)
        print("Driver:", src.driver)
        print("CRS:", src.crs)
        # Si quieres, imprime la resolución espacial (tamaño de píxel)
        print("Tamaño de píxel (x, y):", src.res)

hr_path = "Datasets/SatImages/perusat_v5/LR_BICUBIC/X2/IMG_PER1_20161203152919_ORT_MS_000670_1024-512_x2.tif"
inspect_raster(hr_path)

Ruta del archivo: Datasets/SatImages/perusat_v5/LR_BICUBIC/X2/IMG_PER1_20161203152919_ORT_MS_000670_1024-512_x2.tif
Número de bandas: 4
Anchura (px): 256
Altura (px): 256
Driver: GTiff
CRS: None
Tamaño de píxel (x, y): (1.0, 1.0)
